In [ ]:
import csv
import math

import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

########################Parte di codice commentata utilizzata per generare i dati di latitudine e longitudine normalizzati

with open("/home/studente1/Downloads/aspis_out.csv",'r') as file:
    reader = csv.DictReader(file)
    total0_lat=0
    total0_lon=0
    count0=0
    for row in reader:
        total0_lat+= float(row['latitude'])
        total0_lon+= float(row['longitude'])
        count0+=1
    file.seek(0)
    next(reader)
    mean0_lat = total0_lat / count0
    mean0_lon = total0_lon / count0
    variance_lat0=0
    variance_lon0 = 0
    for row in reader:
        value_lat0 = float(row['latitude'])
        variance_lat0+=(value_lat0-mean0_lon) ** 2
        value_lon0 = float(row['longitude'])
        variance_lon0 += (value_lon0 - mean0_lon) ** 2
    variance_lat0= variance_lat0 /(count0-1)
    variance_lon0 = variance_lon0 / (count0 - 1)
    std_dev_lat0 = math.sqrt(variance_lat0)
    std_dev_lon0 = math.sqrt(variance_lon0)

with open("/home/studente1/Downloads/aspis_out.csv",'r') as file:
    reader= csv.DictReader(file)
    rows0=[]
    for row in reader:
        lat_value0 = float(row['latitude'])
        lon_value0 = float(row['longitude'])
        lat_z0 = (lat_value0 - mean0_lat) / std_dev_lat0
        lon_z0 = (lon_value0 - mean0_lon) / std_dev_lon0
        row['latitude-norm'] = lat_z0
        row['longitude-norm'] = lon_z0
        rows0.append(row)

with open("/home/studente1/Downloads/aspis_out.csv", 'w', newline='') as file:
    fieldnames = reader.fieldnames + ['latitude-norm','longitude-norm']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows0:
        writer.writerow(row)

with open("/home/studente1/Downloads/berus_out.csv", 'r') as file:
    reader = csv.DictReader(file)
    total1_lat = 0
    total1_lon = 0
    count1 = 0
    for row in reader:
        total1_lat += float(row['latitude'])
        total1_lon += float(row['longitude'])
        count1 += 1
    file.seek(0)
    next(reader)
    mean1_lat = total1_lat / count1
    mean1_lon = total1_lon / count1
    variance_lat=0
    variance_lon = 0
    for row in reader:
        value_lat = float(row['latitude'])
        variance_lat+=(value_lat-mean1_lon) ** 2
        value_lon = float(row['longitude'])
        variance_lon += (value_lon - mean1_lon) ** 2
    variance_lat= variance_lat /(count1-1)
    variance_lon = variance_lon / (count1 - 1)
    std_dev_lat1 = math.sqrt(variance_lat)
    std_dev_lon1 = math.sqrt(variance_lon)

with open("/home/studente1/Downloads/berus_out.csv",'r') as file:
    reader= csv.DictReader(file)
    rows1=[]
    for row in reader:
        lat_value = float(row['latitude'])
        lon_value = float(row['longitude'])
        lat_z = (lat_value - mean1_lat) / std_dev_lat1
        lon_z = (lon_value - mean1_lon) / std_dev_lon1
        row['latitude-norm'] = lat_z
        row['longitude-norm'] = lon_z
        rows1.append(row)

with open("/home/studente1/Downloads/berus_out.csv", 'w', newline='') as file:
    fieldnames = reader.fieldnames + ['latitude-norm','longitude-norm']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows1:
        writer.writerow(row)

########################

# Carica i dati della prima classe dal primo file CSV
class1_data = pd.read_csv("/home/studente1/Downloads/aspis_out.csv")
class1_data['label'] = 0

# Carica i dati della seconda classe dal secondo file CSV
#Forse può creare problemi il fatto che
# i due dataset siano di grandezze molto differenti (2000vs6000)
class2_data = pd.read_csv("/home/studente1/Downloads/berus_out.csv")
class2_data['label'] = 1

# Unisci i dati delle due classi in un unico DataFrame
data = pd.concat([class1_data, class2_data], ignore_index=True)

# Seleziona le colonne di interesse
X = data[['latitude-norm', 'longitude-norm']]
y = data['label']

# Effettua lo split dei dati in set di training e di testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#print(len(X_train))
#print(len(y_train))
#print(len(X_test))
#print(len(y_test))
#print(X_train)
#print(y_train)
#print(X_test)
#print(y_test)
# Addestra un modello di classificazione sui dati di training

model = LogisticRegression()
model.fit(X_train, y_train)

# Valuta il modello sui dati di testing
y_pred = model.predict(X_test)
print("Accuracy LogisticRegression:", roc_auc_score(y_test,model.predict(X_test)))

#Addestro il modello e lo valuto con la curva di roc
clf= svm.SVC(kernel='linear')
clf.fit(X_train,y_train)
print("Accuracy SVM linear:", roc_auc_score(y_test,clf.predict(X_test)))

import plotly.express as px
import pandas as pd
#fig = px.scatter_geo(data,lat="latitude",lon="longitude",color='label',color_discrete_map={"0":"blue", "1":"yellow"})
#fig.update_layout(title = 'dataset vipera aspis e berus', title_x=0.5)# legend_title="Legenda", showlegend=True)
#fig.show()
